# SPARQL: Die RDF Abfragesprache

In dieser Übung schauen wir uns die SPARQL Abfragesprache für RDF in der Praxis an. Wir verwenden dafür etwas RDF mit der Pink Floyd Diskographie und führen einige Abfragen aus. Zum Schluss schreiben Sie dann Ihr eigenes RDF und einige SPARQL Abfragen dazu.

In [1]:
!pip install rdflib

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
from io import BytesIO, StringIO
from rdflib import Graph
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer
from IPython.display import display

rdf = """
@prefix ex: <http://example.org#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a ex:Album ;
   ex:title "The Dark Side of the Moon"^^xsd:string ;
   ex:label "Harvest, EMI"@en ;
   ex:released [ 
     ex:day "16"^^xsd:int ;
     ex:month "03"^^xsd:int ;
     ex:year "1973"^^xsd:int 
   ] .
   
[] a ex:Album ;
   ex:title "The Wall" ;
   ex:label "Harvest, EMI" ;
   ex:released [ 
     ex:day 30 ;
     ex:month "11"^^xsd:string ;
     ex:year "1979"^^xsd:int 
   ] .

[] a ex:Single ;
   ex:title "What God Wants, Part 1"^^xsd:string ;
   ex:author [
     ex:firstname "Roger" ;
     ex:lastname "Waters"
   ] ;
   ex:released [ 
     ex:year "1992"^^xsd:int 
   ] .
"""

g = Graph()

r = g.parse(data=rdf, format='turtle')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(StringIO(output.getvalue().decode())))

Führen Sie nun die folgenden Abfragen aus und beantworten Sie die Fragen.

In [3]:
query("""
SELECT ?title WHERE { 
  ?work ex:title ?title
}
""")

,title
0,The Wall
1,The Dark Side of the Moon
2,"What God Wants, Part 1"


In [4]:
query("""
SELECT ?work ?title WHERE { 
  ?work ex:title ?title
}
""")

,work,title
0,ub1bL14C1,The Wall
1,ub1bL5C1,The Dark Side of the Moon
2,ub1bL23C1,"What God Wants, Part 1"


In [5]:
query("""
SELECT ?title WHERE { 
  [] ex:title ?title
}
""")

# Was ist der Unterschied zum vorherigem Beispiel? Antwort: Hier wurde nur nach den Predicaten namens 'ex:title' mit der Variable als Object gefiltert. Oben wird auch nach 'work' gesucht.

,title
0,The Dark Side of the Moon
1,The Wall
2,"What God Wants, Part 1"


In [6]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ; 
     ex:title ?title
}
""")

# Warum nur zwei Resultate? Antwort: Weil bei dem ersten graph pattern nach dem Object Album gefiltert wird.

,title
0,The Dark Side of the Moon
1,The Wall


In [7]:
query("""
SELECT ?s ?p ?o WHERE { 
  ?s ?p ?o
}
""")

# Was erhält man hier? Antwort: Die gesamte Tripelmenge der RDF

,s,p,o
0,ub1bL23C1,http://example.org#title,"What God Wants, Part 1"
1,ub1bL5C1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org#Album
2,ub1bL17C16,http://example.org#day,30
3,ub1bL23C1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org#Single
4,ub1bL8C16,http://example.org#year,1973
5,ub1bL14C1,http://example.org#label,"Harvest, EMI"
6,ub1bL23C1,http://example.org#author,ub1bL25C14
7,ub1bL5C1,http://example.org#label,"Harvest, EMI"
8,ub1bL8C16,http://example.org#month,3
9,ub1bL25C14,http://example.org#firstname,Roger


In [ ]:
query("""
SELECT ?title ?year WHERE { 
  [] rdf:type ex:Album ; 
     ex:title ?title ;
     ex:released [ ex:year ?year ]
  FILTER (?year > 1973)
}
""")

In [ ]:
query("""
SELECT ?title ?year WHERE { 
  {
    [] rdf:type ex:Album ; 
       ex:title ?title ;
       ex:released [ ex:year ?year ]
    FILTER (?year > 1973)
  }
  UNION
  {
    [] rdf:type ex:Single ; 
       ex:title ?title ;
       ex:released [ ex:year ?year ]
    FILTER (?year <= 2000 )
  }
}
""")

In [8]:
query("""
SELECT ?title ?label WHERE { 
    ?work ex:title ?title .
    OPTIONAL { ?work ex:label ?label }
}
""")

# Warum ist der label von "What God Wants, Part 1" NaN? Antwort: Weil die Filterung nach dem Predicat 'ex:label' mit dem Object als Variable nur optional ist. Die letzte RDF hat aber kein Object mit dem Namen 'label'. 

,title,label
0,The Wall,"Harvest, EMI"
1,The Dark Side of the Moon,"Harvest, EMI"
2,"What God Wants, Part 1",NaN


In [ ]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:label ?label
  FILTER (LANG(?label) = "en")
}
""")

In [ ]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:released [ ex:day ?day ]
  FILTER (?day > 15)
}
""")

In [9]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:released [ ex:month ?month ]
  FILTER (DATATYPE(?month) = xsd:string)
}
""")

# Warum erhält man hier nur "The Wall" als Resultat? Antwort: Da bei dem ersten RDF Dokument das Object 'month' den Datetyp int hat. Hier ist aber nicht dem Datentyp string gefiltert.

,title
0,The Wall


Schreiben Sie nun ihr eigenes RDF und werten Sie einige SPARQL Abfragen aus.

In [10]:
rdf = """
<rdf:Description rdf:about="http://example.org#Einkaufsliste">
    <ex:Backzutaten rdf:parseType="Collection">
        <rdf:Description rdf:about="http://www.example.org/Brot"/>
        <rdf:Description rdf:about="http://www.example.org/Mehl" />
        <rdf:Description rdf:about="http://www.example.org/Hefe" />
        <rdf:Description rdf:about="http://www.example.org/Olivenöl" />
        <rdf:Description rdf:about="http://www.example.org/Salz" />
    </ex:Backzutaten>
</rdf:Description>
"""

g = Graph()

r = g.parse(data=rdf, format='turtle')

rdf:Description rdf:about="http://example.org#Einkaufsliste" does not look like a valid URI, trying to serialize this will break.
ex:Backzutaten rdf:parseType="Collection" does not look like a valid URI, trying to serialize this will break.
rdf:Description rdf:about="http://www.example.org/Brot"/ does not look like a valid URI, trying to serialize this will break.


BadSyntax: at line 5 of <>:
Bad syntax (expected '.' or '}' or ']' at end of statement) at ^ in:
"...b'scription rdf:about="http://www.example.org/Brot"/>\n        '^b'<rdf:Description rdf:about="http://www.example.org/Mehl" />\n'..."

In [ ]:
query("""
... Meine SPARQL Abfrage ...
""")